In [ ]:
import os 
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import h5py
import numpy as np 


from scipy.integrate import simps
from scipy.optimize import curve_fit
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.lines import Line2D

sns.set_context("talk")

%matplotlib inline

In [ ]:


# Fit the resolution data to a polynomial for extrapolation
def poly_func(x, a, b, c):
    return a * x**2 + b * x + c

# Create a function to get resolution for any energy
def get_resolution(e,popt):
    return np.maximum(poly_func(e, *popt), 1e-10)  # Ensure non-zero resolution


# Perform Gaussian broadening using convolution with energy-dependent widths
def gaussian_broadening(dos, energy, sigmas):
    broadened_dos = np.zeros_like(dos)
    for i in range(len(energy)):
        # Create a Gaussian kernel centered at each energy point with width sigmas[i]
        kernel = np.exp(-0.5 * ((energy - energy[i]) / sigmas[i])**2)
        kernel /= simps(kernel, energy)  # Normalize the kernel to preserve area
        broadened_dos += dos[i] * kernel  # Convolve DOS with Gaussian kernel
    return broadened_dos

def apply_gaussian_broadening(dos_data,metric='dos total (nm2/ps)',unit_cell_atoms=1):
    energy = dos_data['Energy Transfer (meV)'].values
    dos = dos_data[metric].values

    resolution_data = pd.read_csv('../data/experiment/resolution.csv')
    res_energy = resolution_data['Energy (meV)'].values
    resolution = resolution_data['Resolution'].values
    popt, _ = curve_fit(poly_func, res_energy, resolution)
    
    # Create an array of sigmas for Gaussian broadening based on the resolution function
    sigmas = get_resolution(energy,popt) / np.sqrt(8 * np.log(2))  # Convert FWHM to sigma


    broadened_dos = gaussian_broadening(dos, energy, sigmas)

    # Area normalize both original and broadened DOS
    original_area = simps(dos, energy)
    broadened_area = simps(broadened_dos, energy)

    dos_normalized = dos / original_area
    broadened_dos_normalized = broadened_dos / broadened_area
    return 3*unit_cell_atoms*broadened_dos_normalized

In [ ]:
results_directory={"../results/lammps/nanoparticle/2nm/lennard_jones/results/mdanse/":"LJ" 
                  }
df_list=list()

min_energy=0.25
max_energy=150

for data_directory,force_field in results_directory.items():
    for filename in os.listdir(data_directory):
        if not "dos.h5" in filename:
            continue
        print(os.path.join(data_directory,filename))
        h5file=h5py.File(os.path.join(data_directory,filename))
        data_dict=dict()
        omega_shape=h5file["omega"][:].shape
        for k in h5file.keys():
            data_array=h5file[k][:]
            if data_array.shape != omega_shape or "_0" in k:
                continue
            units=h5file[k].attrs['units']
            data_dict["%s (%s)"%(k.replace("_"," "),units)]=data_array
        df_tmp=pd.DataFrame(data_dict).query("%f < `omega (rad/ps)` and `omega (rad/ps)` < %f"%(min_energy/0.6582,max_energy/0.6582))
        directory_split=data_directory.split("/")
        system=directory_split[3]
        df_tmp["Temperature (K)"]=int(filename.split("_")[2].replace('K',''))
        df_tmp["Weight"]=" ".join(filename.split("_")[:-2]).replace('b ','').capitalize()
        label="%s lammps"%(system.replace("_"," ").replace("nanoparticle", "np"))
        df_tmp["Method"]="LAMMPS"
        df_tmp["Force Field"]=force_field
        df_tmp["Energy Transfer (meV)"]=df_tmp["omega (rad/ps)"]*0.6582
        df_tmp["Normalized Count"]=apply_gaussian_broadening(df_tmp,unit_cell_atoms=137)
        for species,unit_cell_atoms in {"Fe":105-32,"O":32}.items():
            df_tmp[species]=apply_gaussian_broadening(df_tmp,"dos %s (nm2/ps)"%species,unit_cell_atoms)
            df_tmp["Max Normalized %s"%species]=df_tmp[species]/df_tmp[species].max()
        df_tmp["Max Normalized Count"]=df_tmp["Normalized Count"]/df_tmp["Normalized Count"].max()
        df_tmp["Radius (nm)"]=int(directory_split[4].replace("nm",""))
        df_tmp["Software"]=directory_split[2]
        df_tmp["System"]=system #.split("_")[0].replace("nanoparticle", "np")
        
        df_list.append(df_tmp)
df_dos_lammps=pd.concat(df_list,ignore_index=True)

df_dos_lammps

In [ ]:
results_directory={"../results/gromacs/nanoparticle/2nm/results/md_phonon_200/":200
                   ,"../results/gromacs/nanoparticle/2nm/results/md_phonon_300/":300
                   
                  }
df_list=list()

min_energy=0.25
max_energy=150

for data_directory,temperature in results_directory.items():
    for filename in os.listdir(data_directory):
        if not "dos.h5" in filename:
            continue
        print(os.path.join(data_directory,filename))
        h5file=h5py.File(os.path.join(data_directory,filename))
        data_dict=dict()
        omega_shape=h5file["omega"][:].shape
        for k in h5file.keys():
            data_array=h5file[k][:]
            if data_array.shape != omega_shape or "_0" in k:
                continue
            units=h5file[k].attrs['units']
            data_dict["%s (%s)"%(k.replace("_"," "),units)]=data_array
        df_tmp=pd.DataFrame(data_dict).query("%f < `omega (rad/ps)` and `omega (rad/ps)` < %f"%(min_energy/0.6582,max_energy/0.6582))
        directory_split=data_directory.split("/")
        system=directory_split[3]
        df_tmp["Temperature (K)"]=temperature
        df_tmp["Weight"]=" ".join(filename.split("_")[:-1]).replace('b ','').capitalize()
        df_tmp["Method"]="GROMACS"
        df_tmp["Force Field"]="LJ"
        df_tmp["Energy Transfer (meV)"]=df_tmp["omega (rad/ps)"]*0.6582
        df_tmp["Normalized Count"]=apply_gaussian_broadening(df_tmp,unit_cell_atoms=137)
        for species,unit_cell_atoms in {"Fe":105-32,"O":32}.items():
            df_tmp[species]=apply_gaussian_broadening(df_tmp,"dos %s (nm2/ps)"%species,unit_cell_atoms)
            df_tmp["Max Normalized %s"%species]=df_tmp[species]/df_tmp[species].max()
        df_tmp["Max Normalized Count"]=df_tmp["Normalized Count"]/df_tmp["Normalized Count"].max()
        df_tmp["Radius (nm)"]=int(directory_split[4].replace("nm",""))
        df_tmp["Software"]=directory_split[2]
        df_tmp["System"]=system #.split("_")[0].replace("nanoparticle", "np")
        
        df_list.append(df_tmp)
        
df_dos_gromacs=pd.concat(df_list,ignore_index=True)
df_dos_gromacs

In [ ]:
def read_timeseries(filename):
    print("oppening %s"%filename)
    h5file=h5py.File(filename)
    data_dict=dict()
    for k in h5file.keys():
        units=h5file[k].attrs['units']
        data_dict["%s (%s)"%(k.replace("_"," "),units)]=h5file[k][:]
    df_tmp=pd.DataFrame(data_dict)
    df_tmp=df_tmp.drop(columns=[x for x in df_tmp.columns if "group" in x])
    df_tmp['time (ns)']=df_tmp['time (ps)']*0.001
    return df_tmp

input_dict={"../results/lammps/nanoparticle/2nm/lennard_jones/results/mdanse/nvt_prod_200K_rmsd.h5":"LAMMPS"
           ,"../results/gromacs/nanoparticle/2nm/results/md_prod_200/rmsd.h5":"GROMACS"
           }

df_list=list()
for input_file,software in input_dict.items():
    df_tmp=read_timeseries(input_file)
    df_tmp["Software"]=software
    df_list.append(df_tmp)
df_rmsd=pd.concat(df_list,ignore_index=True)
df_rmsd

In [ ]:
input_dict={"../results/lammps/nanoparticle/2nm/lennard_jones/results/mdanse/nvt_prod_200K_ecc.h5":"LAMMPS"
           ,"../results/gromacs/nanoparticle/2nm/results/md_prod_200/ecc.h5":"GROMACS"
           }

df_list=list()
for input_file,software in input_dict.items():
    df_tmp=read_timeseries(input_file)
    df_tmp["Software"]=software
    df_list.append(df_tmp)
df_ecc=pd.concat(df_list,ignore_index=True)
df_ecc

In [ ]:
input_dict={"../results/lammps/nanoparticle/2nm/lennard_jones/results/nvt_prod_200K_rog/rog.csv":"LAMMPS"
           ,"../results/gromacs/nanoparticle/2nm/results/md_prod_200/rog/rog.csv":"GROMACS"
           }

df_list=list()
for input_file,software in input_dict.items():
    df_tmp=pd.read_csv(input_file)
    df_tmp["Software"]=software
    df_list.append(df_tmp)
df_rog=pd.concat(df_list,ignore_index=True)
df_rog

In [ ]:
fig, axs = plt.subplots(ncols=2,nrows=2,figsize=(7*2,4*2))
sns.set_context("talk")

labx=0.9
laby=1
emin,emax=0.25,150
index=["(%s)"%x for x in list("abcd")]


df_dos=pd.concat([df_dos_lammps.query(" %f < `Energy Transfer (meV)` and `Energy Transfer (meV)` < %f and Weight != 'Atomic weight'  and `Temperature (K)` == 200 and `Force Field` == 'LJ' "%(emin,emax)),
                  df_dos_gromacs.query(" %f < `Energy Transfer (meV)` and `Energy Transfer (meV)` < %f and `Temperature (K)` == 200"%(emin,emax))
                 ],ignore_index=True)

df_dos["Software"]=df_dos["Software"].apply(lambda x : x.upper())

for i, k1 in enumerate(np.reshape([
    {"data":df_dos,"x":'Energy Transfer (meV)',"y":'Normalized Count',"y_label":"PDOS (1/meV)","x_minor_tick":25/2,"y_minor_tick":1}
    ,{"data":df_rog,"x":'Time (ps)',"y":'Radius (nm)',"y_label":"Radius of Gyration ($nm$)","x_minor_tick":100,"y_minor_tick":0.0005}
    ,{"data":df_rmsd,"x":'time (ps)',"y":'rmsd total (nm)',"y_label":"Root Mean\nSquare Deviation ($nm$)","x_minor_tick":100,"y_minor_tick":0.01}
    ,{"data":df_ecc,"x":'time (ps)',"y":'eccentricity (unitless)',"y_label":"Eccentricity","x_minor_tick":100,"y_minor_tick":0.0005}

],(2,2))):
    for j,v in enumerate(k1):

        ax=axs[i,j]
        ax.text(labx, laby, index[2*i+j],weight='bold' ,transform=ax.transAxes)
        sns.lineplot(data=v["data"]
                    ,y=v["y"]
                    ,x=v["x"]
                    ,ax=ax
                    ,style="Weight" if i+j==0 else None
                   ,hue="Software"
                    ,legend=i+j==0 
               )


        ax.spines['bottom'].set_color('0')
        ax.spines['top'].set_color('1')
        ax.spines['right'].set_color('1')
        ax.spines['left'].set_color('0')
        ax.tick_params(direction='out', width=3, bottom=True, left=True)
        ax.grid(False)
        ax.set_ylabel(v["y_label"])
        if i+j==0 :
            sns.move_legend(ax,'center',bbox_to_anchor=(0.5,0.975),ncol=6
                            ,bbox_transform=fig.transFigure
                           )


        ax.xaxis.set_minor_locator(MultipleLocator(v["x_minor_tick"]))
        ax.yaxis.set_minor_locator(MultipleLocator(v["y_minor_tick"]))
        ax.tick_params(which='minor', length=4)


# set the spacing between subplots
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.95, 
                    top=0.9, 
                    wspace=0.25, 
                    hspace=0.4
                   )

plt.savefig("../figures/fig3.pdf", pad_inches=0.2,bbox_inches="tight")